In [1]:
from typing import List
from os import path
from re import findall
from time import sleep
from tqdm import tqdm

import pickle
import json
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
CLIENTS_FILE = path.join('clients/', '20220106_clients.pkl')
HTML_FILE = path.join('htmls/', '20220309_continua-summer-sale.html')

In [3]:
def load_auth(file_path: path) -> dict:
    with open(file_path, 'r') as f:
        return json.load(f)

def load_clients(file_path: path) -> dict:
    with open(file_path, 'rb') as f:
        return pickle.load(f)

def load_html(file_path: path) -> str:
    with open(file_path, 'r') as f:
        return f.read()

def extract_subject(base_html: str) -> str:
    return findall('<title.*?>(.+?)</title>', base_html)[0]
    
def compose_body(base_html: str, client_name: str) -> MIMEText:
    name = f'{client_name.strip().split()[0]}'
    body = base_html.replace('#$#CLIENTNAME', name)
    return MIMEText(body, 'html')

def compose_message(message_from: str, message_to: str, message_subject: str, message_body: MIMEText) -> MIMEMultipart:
    message = MIMEMultipart()
    message['From'] = message_from
    message['To'] = message_to
    message['Subject'] = message_subject
    message.attach(message_body)
    return message

def mount_server(user: str, password: str) -> smtplib.SMTP_SSL:
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(user, password)
        print(' [✓] Succesful connection to server.')
        return server
    except Exception as err:
        print(f' [×] Something went wrong...\n --- {err}')
    
def send_unitary_email(auth: dict, base_html: str, recipient_name: str, recipient_email: str, subject: str, waiting_period=15) -> None:
    user, pswd, gkey = auth.values()
    body = compose_body(base_html, recipient_name)
    mssg = compose_message(user, recipient_email, subject, body)
    try:
        server = mount_server(user, gkey)
        server.send_message(mssg)
        print(' [✓] Email delivered!')
    except Exception as err:
        print(f' [×] Failure to deliver email to recipient.\n --- {err}\n --- Trying again in {waiting_period} seconds...')
        sleep(waiting_period)
        send_unitary_email(auth, base_html, recipient_name, recipient_email, subject)
        
def send_bulk_emails(auth: dict, clients: dict, base_html: str, subject: str) -> None:
    i = 0
    for recipient_name, recipient_email in clients.items():
        i += 1
        print(f'Sending email to {recipient_name} · {(i/len(clients)*100):.2f}% · {i}/{len(clients)}')
        send_unitary_email(auth, base_html, recipient_name, recipient_email, subject)
        
def send_test_email(auth: dict, base_html: str, subject: str) -> None:
    recipient_name = 'CAMPAIGN-TEST'
    recipient_email = auth['USER']
    send_unitary_email(auth, base_html, recipient_name, recipient_email, subject)

In [4]:
auth = load_auth(path.join('auth.json'))
base_html = load_html(HTML_FILE)
subject = extract_subject(base_html)
clients = load_clients(CLIENTS_FILE)
clients.update({'AMIGAS-VNTG': auth['USER']})

In [5]:
send_test_email(auth, base_html, subject)

 [✓] Succesful connection to server.
 [✓] Email delivered!


In [6]:
# Bulk processing of campaign emails.
send_bulk_emails(auth, clients, base_html, subject)

Sending email to Adriana Latorre Carvallo · 0.35% · 1/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Akarimm · 0.70% · 2/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Akza Ugarte · 1.05% · 3/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Aparicio · 1.40% · 4/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Campos · 1.75% · 5/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Espinoza García · 2.10% · 6/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Ortega · 2.45% · 7/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Oyarzún Vargas · 2.80% · 8/286
 [✓] Succesful connection to server.
 [✓] Email delivered!
Sending email to Alejandra Rojas Calderón · 3.15% · 9/286
 [✓] Succesful connection to server.
 [✓] Email delivered